In [1]:
from sklearn.datasets import fetch_openml

bunch = fetch_openml('mushroom')
bunch.frame

,cap-shape,cap-surface,cap-color,bruises%3F,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,k,s,n,f,n,a,c,b,y,e,...,o,o,p,o,o,p,b,c,l,e
8120,x,s,n,f,n,a,c,b,y,e,...,o,o,p,n,o,p,b,v,l,e
8121,f,s,n,f,n,a,c,b,n,e,...,o,o,p,o,o,p,b,c,l,e
8122,k,y,n,f,y,f,c,n,b,t,...,w,w,p,w,o,e,w,v,l,p


In [2]:
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

frame = bunch.frame.dropna()
names = frame.columns.tolist()

data = pd.get_dummies(frame)
train, test = train_test_split(data, stratify = data[data.columns[-1]], random_state = 0)
print (len(train), len(test))

4233 1411


In [5]:
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth, association_rules

frequent_itemsets = fpgrowth(train, use_colnames=True, min_support = 0.5)
# frequent_itemsets
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold = 0.7)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(veil-type_p),(veil-color_w),1.000000,0.998583,0.998583,0.998583,1.000000,0.000000,1.000000
1,(veil-color_w),(veil-type_p),0.998583,1.000000,0.998583,1.000000,1.000000,0.000000,inf
2,(veil-type_p),(gill-attachment_f),1.000000,0.996456,0.996456,0.996456,1.000000,0.000000,1.000000
3,(gill-attachment_f),(veil-type_p),0.996456,1.000000,0.996456,1.000000,1.000000,0.000000,inf
4,(veil-color_w),(gill-attachment_f),0.998583,0.996456,0.995039,0.996451,0.999995,-0.000005,0.998583
...,...,...,...,...,...,...,...,...,...
10700,"(stalk-color-below-ring_w, veil-color_w)","(veil-type_p, ring-number_o, gill-attachment_f)",0.553744,0.971179,0.534137,0.964590,0.993216,-0.003648,0.813938
10701,"(veil-type_p, stalk-color-below-ring_w)","(ring-number_o, gill-attachment_f, veil-color_w)",0.553744,0.969761,0.534137,0.964590,0.994668,-0.002863,0.853967
10702,"(stalk-color-below-ring_w, gill-attachment_f)","(veil-type_p, ring-number_o, veil-color_w)",0.553744,0.969761,0.534137,0.964590,0.994668,-0.002863,0.853967
10703,"(ring-number_o, stalk-color-below-ring_w)","(veil-type_p, gill-attachment_f, veil-color_w)",0.534137,0.995039,0.534137,1.000000,1.004986,0.002650,inf


In [7]:
confidences = []
for i, row in rules.iterrows():
    acondition = None
    for name in row.antecedents:
        if acondition is None:
            acondition = (test[name] == 1)
        else:
            acondition = acondition & (test[name] == 1)
    
    ccondition = None
    for name in row.consequents:
        if ccondition is None:
            ccondition = (test[name] == 1)
        else:
            ccondition = ccondition & (test[name] == 1)
    
    confidences.append (len(test[acondition & ccondition]) / len(test[acondition]))

print ("test condidence = %.2f, alen = %.2f, clen = %.2f" % (np.mean(confidences), 
                                                            rules.antecedents.apply(len).mean(), 
                                                             rules.consequents.apply(len).mean()))

test condidence = 0.91, alen = 2.87, clen = 2.43


In [8]:
rules.groupby(by='antecedents').count()

,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
antecedents,,,,,,,,
(veil-type_p),31,31,31,31,31,31,31,31
"(gill-attachment_f, gill-spacing_c, veil-type_p, bruises%3F_t, ring-type_p)",9,9,9,9,9,9,9,9
"(gill-attachment_f, veil-color_w, veil-type_p, bruises%3F_t, ring-type_p)",10,10,10,10,10,10,10,10
"(gill-spacing_c, veil-color_w, veil-type_p, bruises%3F_t, ring-type_p)",9,9,9,9,9,9,9,9
"(ring-number_o, bruises%3F_t)",63,63,63,63,63,63,63,63
...,...,...,...,...,...,...,...,...
"(stalk-surface-below-ring_s, gill-spacing_c, veil-color_w, gill-attachment_f, ring-number_o)",3,3,3,3,3,3,3,3
"(stalk-surface-below-ring_s, gill-size_b, gill-attachment_f, veil-color_w)",1,1,1,1,1,1,1,1
"(stalk-surface-below-ring_s, gill-size_b, veil-type_p, veil-color_w)",1,1,1,1,1,1,1,1
